In [ ]:
# https://pypi.python.org/pypi/cirq
!apt-get -qq install -y graphviz && pip install cirq
import cirq
from cirq import X, H, Z, inverse, CX

In [ ]:
def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle1():
    # check 0-2 edge and store at 4th qubit
    yield edge_check(0, 1, 4)
    # check 0-2 edge and store at 5th qubit
    yield edge_check(0, 2, 5)
    # check 0-3 edge and store at 6th qubit
    yield edge_check(0, 3, 6)
    # check 1-3 edge and store at 7th qubit
    yield edge_check(1, 3, 7)

    # check all edge qubits
    yield X(qq[8]).controlled_by(*(qq[4:8]))

def oracle_computation1():
    yield oracle1()
    yield Z(qq[8])
    yield inverse(oracle1())
       
def oracle2():
    # check 0-2 edge and store at 4th qubit
    yield edge_check(0, 2, 4)
    # check 0-3 edge and store at 5th qubit
    yield edge_check(0, 3, 5)
    # check 1-2 edge and store at 6th qubit
    yield edge_check(1, 2, 6)
    # check 1-3 edge and store at 7th qubit
    yield edge_check(1, 3, 7)

    # check all edge qubits
    yield X(qq[8]).controlled_by(*(qq[4:8]))    
    
def oracle_computation2():
    yield oracle1()
    yield Z(qq[8])
    yield inverse(oracle2())    

def inversion():
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[3]).controlled_by(*(qq[0:3]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [ ]:
circuit = cirq.Circuit()

qq = cirq.LineQubit.range(9)

circuit.append(H.on_each(*(qq[0:4])))
##################################################


for i in range(2):
    circuit.append(oracle_computation1())
    circuit.append(inversion())

# we are only intertested in outputs of first 4 qubits
circuit.append(cirq.measure(*(qq[0:4]), key='result'))
######################################################################################################
# determine the statistics of the measurements
trials_number = 1000
s = cirq.Simulator()
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)
print(counts)
# print(circuit)

Counter({'1100': 78, '0100': 77, '0110': 71, '0001': 67, '1001': 67, '0111': 66, '1110': 65, '1111': 63, '0000': 62, '1010': 60, '0101': 57, '1101': 57, '0010': 56, '1011': 53, '1000': 51, '0011': 50})


In [ ]:
circuit = cirq.Circuit()

qq = cirq.LineQubit.range(9)

circuit.append(H.on_each(*(qq[0:4])))
##################################################


for i in range(2):
    circuit.append(oracle_computation2())
    circuit.append(inversion())

# we are only intertested in outputs of first 4 qubits
circuit.append(cirq.measure(*(qq[0:4]), key='result'))
######################################################################################################
# determine the statistics of the measurements
trials_number = 1000
s = cirq.Simulator()
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)
print(counts)
# print(circuit)

Counter({'1100': 252, '0011': 228, '1110': 60, '0001': 58, '0110': 56, '1001': 51, '1011': 49, '0100': 43, '1111': 33, '0101': 29, '0000': 26, '1010': 25, '1101': 23, '1000': 23, '0111': 22, '0010': 22})
